<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=ea04b6243ff8de8f06fac8e2af3e7462830321eaf10c14cc82caecef0d7fd70a
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-14 16:20:11
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.38 C
-------------------
Today PnL: -1.95 L (-1.39%)
Current PnL: -23.19 L (-15.45%)
CY Booked + Current PnL: -11.29 L (-7.52%)
-------------------
Total profit:  1.13 L
Total loss:  -24.31 L
-------------------
Total Booked + Current PnL: 15.69 L (12.75%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.6%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 90.39 L (65.36%)
Deployed:  1.23 C
Current:  1.38 C
CAGR/XIRR %: 6.92%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1662.74,-20.47,55.0,X-LC,3.74,212665.0,8165.0,15120.0,-0.72,3.99,7.11,11.39,10.0,0.54,1.55,13.30,X40N,ATH,PHARMA
76,TTKPRESTIG,770.00,96.16,49.0,M-SC,1.31,84547.0,-16230.0,16326.0,1.25,-16.10,19.31,0.09,245.0,-0.99,0.62,11.08,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,-0.52,53.0,X-MC,1.04,208755.0,17013.0,20750.0,0.96,8.87,9.94,19.69,99.0,0.82,1.52,16.27,XY25,NTT,AC
56,RELIANCE,1533.00,-14.41,48.0,X-LC,2.85,214640.0,4274.0,24512.0,0.07,2.03,11.42,13.68,37.0,0.17,1.56,18.89,XY25,NTT,REFINERIES
41,ITC,452.00,-40.03,38.0,X-LC,2.76,194035.0,-6103.0,26990.0,-0.61,-3.05,13.91,10.44,4.0,-0.23,1.41,2.34,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SONACOMS,1006.0,-36.29,64.0,M-SC,12.86,79240.0,-21919.0,96808.0,3.91,-21.67,122.17,74.03,202.0,-0.23,0.58,11.74,AR,BTT,AUTO
4,ANGELONE,3033.0,11.22,64.0,X-SC,10.32,197177.0,6171.0,51522.0,2.73,3.23,26.13,30.21,157.0,0.12,1.44,24.79,X40N,NTT,FINANCE
7,ATULAUTO,844.0,3689.19,53.0,M-SC,4.80,170119.0,-27013.0,116838.0,1.56,-13.70,68.68,45.57,236.0,-0.23,1.24,20.61,XY24,NTT,AUTO
60,SAMMAANCAP,326.0,-186.36,67.0,M-SC,17.50,148293.0,-927.0,145105.0,1.45,-0.62,97.85,96.62,208.0,-0.01,1.08,60.49,XY25,NTT,FINANCE
85,WIPRO,420.0,-12.43,54.0,M-LC,5.94,154020.0,3075.0,106382.0,1.34,2.04,69.07,72.51,53.0,0.03,1.12,8.18,XR,NTT,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.0,-54.49,11.0,X-LC,1.10,143548.0,-132975.0,243041.0,-40.15,-48.09,169.31,39.81,54.0,-0.55,1.05,0.00,XY24,NTT,AUTO
81,VALIANTORG,838.0,-277.79,23.0,H-SC,3.11,98906.0,-64699.0,186013.0,-4.78,-39.55,188.07,74.15,139.0,-0.35,0.72,26.78,XR,NTT,CHEMICALS
23,DEN,75.0,86.53,36.0,M-SC,3.89,52704.0,-31760.0,67298.0,-4.30,-37.60,127.69,42.07,237.0,-0.47,0.38,11.93,AR,NTT,ENTERTAINMENT
25,GREENPANEL,537.0,210.72,45.0,M-SC,4.38,138787.0,-44291.0,123271.0,-3.66,-24.19,88.82,43.14,230.0,-0.36,1.01,30.55,XY24,NTT,MISC
10,BANDHANBNK,400.0,-12.63,47.0,H-SC,2.96,217871.0,-60688.0,318135.0,-3.50,-21.79,146.02,92.42,151.0,-0.19,1.59,25.91,XY24,NTT,BANKS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-186.36,67.0,M-SC,17.5,148293.0,-927.0,145105.0,1.45,-0.62,97.85,96.62,208.0,-0.01,1.08,60.49,XY25,NTT,FINANCE
36,INDIAMART,4810.62,-55.76,36.0,H-SC,8.3,123744.0,408.0,131218.0,-2.66,0.33,106.04,106.72,119.0,0.00,0.90,23.79,AR,ATH,MISC
54,RECLTD,446.00,45.36,43.0,M-LC,6.2,203665.0,495.0,41629.0,-0.22,0.24,20.44,20.74,55.0,0.01,1.48,5.80,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,103.03,55.0,H-SC,11.52,128466.0,-14013.0,33530.0,-2.38,-9.84,26.10,13.70,163.0,-0.42,0.94,51.63,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,96.16,49.0,M-SC,1.31,84547.0,-16230.0,16326.0,1.25,-16.10,19.31,0.09,245.0,-0.99,0.62,11.08,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.97,48.0,H-SC,2.06,220896.0,-48771.0,85111.0,-0.11,-18.09,38.53,13.47,138.0,-0.57,1.61,12.32,XY24,NTT,PAINTS
17,CERA,9475.0,-23.15,43.0,H-SC,1.97,140093.0,-35810.0,77836.0,1.31,-20.36,55.56,23.89,149.0,-0.46,1.02,20.99,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-25.05,42.0,H-MC,7.03,101365.0,-30470.0,72699.0,-0.32,-23.11,71.72,32.03,98.0,-0.42,0.74,13.54,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,45.36,43.0,M-LC,6.20,203665.0,495.0,41629.0,-0.22,0.24,20.44,20.74,55.0,0.01,1.48,5.80,XY25,NTT,FINANCE
31,HINDZINC,730.22,34.91,67.0,M-LC,8.56,219958.0,14882.0,97683.0,0.02,7.26,44.41,54.89,52.0,0.15,1.60,33.10,X5K,ATH,METALS
85,WIPRO,420.00,-12.43,54.0,M-LC,5.94,154020.0,3075.0,106382.0,1.34,2.04,69.07,72.51,53.0,0.03,1.12,8.18,XR,NTT,IT
78,UNIONBANK,163.00,-13.18,51.0,M-LC,9.50,155769.0,14929.0,29565.0,-2.07,10.60,18.98,31.59,66.0,0.50,1.14,39.90,XY24,NTT,BANKS
11,BANKINDIA,190.00,-25.47,58.0,H-MC,12.66,191173.0,11365.0,101054.0,-2.02,6.32,52.86,62.52,88.0,0.11,1.39,42.56,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,34.91,67.0,M-LC,8.56,219958.0,14882.0,97683.0,0.02,7.26,44.41,54.89,52.0,0.15,1.60,33.10,X5K,ATH,METALS
11,BANKINDIA,190.00,-25.47,58.0,H-MC,12.66,191173.0,11365.0,101054.0,-2.02,6.32,52.86,62.52,88.0,0.11,1.39,42.56,XR,NTT,BANKS
85,WIPRO,420.00,-12.43,54.0,M-LC,5.94,154020.0,3075.0,106382.0,1.34,2.04,69.07,72.51,53.0,0.03,1.12,8.18,XR,NTT,IT
36,INDIAMART,4810.62,-55.76,36.0,H-SC,8.30,123744.0,408.0,131218.0,-2.66,0.33,106.04,106.72,119.0,0.00,0.90,23.79,AR,ATH,MISC
84,WHIRLPOOL,2270.00,-48.74,30.0,M-SC,3.41,86550.0,-4948.0,83703.0,-0.88,-5.41,96.71,86.07,223.0,-0.06,0.63,25.69,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-54.49,11.0,X-LC,1.10,143548.0,-132975.0,243041.0,-40.15,-48.09,169.31,39.81,54.0,-0.55,1.05,0.00,XY24,NTT,AUTO
9,BAJAJHFL,181.50,-16.60,29.0,X-MC,5.52,175857.0,-24649.0,121799.0,-1.37,-12.29,69.26,48.45,90.0,-0.20,1.28,1.41,X40N,ATH,FINANCE
51,QUESS,986.00,-48.21,29.0,X-SC,37.86,51279.0,-13727.0,164657.0,-1.82,-21.12,321.10,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
28,HAVELLS,2069.16,-8.87,29.0,X-MC,1.37,226060.0,-21671.0,96731.0,-1.63,-8.75,42.79,30.30,92.0,-0.22,1.65,3.38,X40,ATH,ELECTRICAL
5,ASIANPAINT,3465.66,-19.21,30.0,X-LC,5.31,203993.0,-47775.0,100977.0,-1.04,-18.98,49.50,21.13,27.0,-0.47,1.49,9.73,X40,ATH,PAINTS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-3.25,43.0,H-LC,0.94,155365.0,-30730.0,78211.0,0.97,-16.51,50.34,25.51,15.0,-0.39,1.13,14.49,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,-0.52,53.0,X-MC,1.04,208755.0,17013.0,20750.0,0.96,8.87,9.94,19.69,99.0,0.82,1.52,16.27,XY25,NTT,AC
72,TATAMOTORS,1065.00,-54.49,11.0,X-LC,1.10,143548.0,-132975.0,243041.0,-40.15,-48.09,169.31,39.81,54.0,-0.55,1.05,0.00,XY24,NTT,AUTO
28,HAVELLS,2069.16,-8.87,29.0,X-MC,1.37,226060.0,-21671.0,96731.0,-1.63,-8.75,42.79,30.30,92.0,-0.22,1.65,3.38,X40,ATH,ELECTRICAL
33,ICICIGI,2260.25,-20.11,43.0,X-MC,1.47,137255.0,1122.0,30004.0,1.17,0.82,21.86,22.86,91.0,0.04,1.00,13.54,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.21,29.0,X-SC,37.86,51279.0,-13727.0,164657.0,-1.82,-21.12,321.10,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
72,TATAMOTORS,1065.00,-54.49,11.0,X-LC,1.10,143548.0,-132975.0,243041.0,-40.15,-48.09,169.31,39.81,54.0,-0.55,1.05,0.00,XY24,NTT,AUTO
9,BAJAJHFL,181.50,-16.60,29.0,X-MC,5.52,175857.0,-24649.0,121799.0,-1.37,-12.29,69.26,48.45,90.0,-0.20,1.28,1.41,X40N,ATH,FINANCE
82,VBL,671.64,-21.70,37.0,X-LC,10.51,281741.0,-34101.0,146759.0,-0.19,-10.80,52.09,35.67,5.0,-0.23,2.05,1.56,X40N,ATH,FMCG
21,COLPAL,3767.14,-5.64,36.0,X-MC,7.68,220240.0,-43125.0,156481.0,-0.85,-16.37,71.05,43.04,84.0,-0.28,1.61,2.25,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-48.21,29.0,X-SC,37.86,51279.0,-13727.0,164657.0,-1.82,-21.12,321.10,232.18,198.0,-0.08,0.37,0.00,XY24,NTT,MISC
55,RELAXO,1176.00,-41.14,34.0,X-SC,3.33,80276.0,-64884.0,140812.0,-1.15,-44.70,175.41,52.31,136.0,-0.46,0.59,7.45,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-37.58,34.0,X-SC,4.36,93309.0,-35361.0,80656.0,-2.16,-27.48,86.44,35.20,219.0,-0.44,0.68,7.10,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-6.94,43.0,X-SC,1.05,105840.0,3836.0,40092.0,-2.28,3.76,37.88,43.06,122.0,0.10,0.77,21.52,X40N,ATH,MISC
32,HONAUT,58357.33,-23.12,44.0,X-SC,2.23,108270.0,-19668.0,66803.0,-0.18,-15.37,61.70,36.84,143.0,-0.29,0.79,10.84,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4406.34,-30.29,43.0,X-LC,12.30,278268.0,-67688.0,135934.0,-1.56,-19.57,48.85,19.73,1.0,-0.50,2.03,2.49,X40,ATH,IT
39,INFY,2275.00,-18.52,51.0,X-LC,7.50,317349.0,4461.0,167211.0,-0.22,1.43,52.69,54.87,3.0,0.03,2.31,8.17,X40,BTT,IT
41,ITC,452.00,-40.03,38.0,X-LC,2.76,194035.0,-6103.0,26990.0,-0.61,-3.05,13.91,10.44,4.0,-0.23,1.41,2.34,X40,NTT,FMCG
82,VBL,671.64,-21.70,37.0,X-LC,10.51,281741.0,-34101.0,146759.0,-0.19,-10.80,52.09,35.67,5.0,-0.23,2.05,1.56,X40N,ATH,FMCG
1,ABB,7934.00,-41.23,41.0,H-LC,7.28,245736.0,-15883.0,135106.0,-0.54,-6.07,54.98,45.57,7.0,-0.12,1.79,4.56,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,80.00,-45.67,49.0,L-SC,28.72,81120.0,-32429.0,72481.0,-0.80,-28.56,89.35,35.27,268.0,-0.45,0.59,98.54,XR,NTT,HOTELS
48,MASFIN,398.61,-20.96,40.0,H-SC,9.92,89880.0,-8100.0,29705.0,-0.75,-8.27,33.05,22.05,152.0,-0.27,0.66,30.65,XR,ATH,FINANCE
77,UJJIVANSFB,60.00,103.03,55.0,H-SC,11.52,128466.0,-14013.0,33530.0,-2.38,-9.84,26.10,13.70,163.0,-0.42,0.94,51.63,OX40N,NTT,BANKS
68,SURYODAY,240.00,57.89,53.0,H-SC,10.46,145411.0,-33660.0,99388.0,-3.27,-18.80,68.35,36.71,135.0,-0.34,1.06,44.28,XR,NTT,BANKS
60,SAMMAANCAP,326.00,-186.36,67.0,M-SC,17.50,148293.0,-927.0,145105.0,1.45,-0.62,97.85,96.62,208.0,-0.01,1.08,60.49,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-186.36,67.0,M-SC,17.50,148293.0,-927.0,145105.0,1.45,-0.62,97.85,96.62,208.0,-0.01,1.08,60.49,XY25,NTT,FINANCE
68,SURYODAY,240.00,57.89,53.0,H-SC,10.46,145411.0,-33660.0,99388.0,-3.27,-18.80,68.35,36.71,135.0,-0.34,1.06,44.28,XR,NTT,BANKS
31,HINDZINC,730.22,34.91,67.0,M-LC,8.56,219958.0,14882.0,97683.0,0.02,7.26,44.41,54.89,52.0,0.15,1.60,33.10,X5K,ATH,METALS
50,PGHH,17907.65,-26.18,61.0,X-MC,2.92,213690.0,12870.0,54918.0,0.47,6.41,25.70,33.76,80.0,0.23,1.56,11.94,X40,ATH,FMCG
4,ANGELONE,3033.00,11.22,64.0,X-SC,10.32,197177.0,6171.0,51522.0,2.73,3.23,26.13,30.21,157.0,0.12,1.44,24.79,X40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.43
1,25,43.94
2,50,75.81


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.53
LC    33.61
MC    21.87
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.70
X40      14.83
XY25     12.10
XR       10.99
X40N     10.10
AR        8.22
OX40N     7.72
X200      1.79
X5K       1.60
SR        1.13
MH        0.83
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.67
X-LC    22.53
X-MC    16.14
M-SC    12.77
M-LC     5.34
H-LC     4.71
X-SC     4.64
H-MC     3.80
M-MC     1.62
L-SC     1.45
L-LC     1.03
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.04,-6.46,41.55
IT,13.04,-17.48,81.11
FINANCE,12.20,-12.99,64.20
MISC,6.91,-21.17,88.32
BANKS,6.43,-13.22,72.78
PAINTS,5.62,-19.85,37.71
ELECTRICAL,5.36,-11.96,51.24
INSURANCE,3.84,-5.00,44.25
AUTO,2.87,-46.30,116.23


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3363823.0,22
XR,1402228.0,14
AR,1200617.0,9
X40,806184.0,10
X40N,641666.0,8
OX40N,588810.0,10
XY25,520887.0,8
SR,267261.0,2
X5K,97683.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3481178.0,25
M-SC,1289377.0,15
X-LC,1147804.0,13
X-MC,1114297.0,11
X-SC,544542.0,6
H-MC,357900.0,3
H-LC,292563.0,3
M-LC,275259.0,4
L-SC,262080.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1213855.0      6
           XR         829616.0      7
           AR         823194.0      5
M-SC       XY24       697442.0      6
X-MC       XY24       590228.0      4
X-LC       X40        500489.0      5
           XY24       313055.0      2
X-SC       X40N       299229.0      4
H-SC       OX40N      276656.0      4
           SR         267261.0      2
X-MC       X40        225039.0      4
X-LC       X40N       220638.0      3
H-LC       AR         213317.0      2
H-MC       XY24       184147.0      1
X-MC       XY25       177231.0      2
L-SC       XR         171876.0      2
M-MC       XY24       170874.0      1
X-SC       XY24       164657.0      1
M-SC       AR         164106.0      2
           OX40N      149251.0      4
           XY25       145105.0      1
           XR         133473.0      2
X-MC       X40N       121799.0      1
X-LC       XY25       113622.0      3
M-LC       XR         106382.0      1
H-MC       XR         101054.0      1
M-LC       X5K         97683.0      1
L-SC       OX40N       90204.0      1
X-SC       X40         80656.0      1
H-LC       X200        79246.0      1
H-MC       OX40N       72699.0      1
H-SC       MH          70596.0      1
L-MC       XR          59827.0      1
L-LC       XY25        43300.0      1
M-LC       XY25        41629.0      1
           XY24        29565.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 34.0 seconds
